In [1]:
import csv
import pandas as pd
from django.contrib.gis.geos import Point
import os
from django.db import connection
from batid.utils.db import dictfetchall
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

df = pd.read_csv("adresses-33.csv", sep=";")


/var/folders/wp/_k1czs557d53n70cnnnzq9280000gp/T/ipykernel_51167/961146359.py:9: DtypeWarning: Columns (5,7,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("adresses-33.csv", sep=";")


In [2]:
print(df.keys())

print(df['certification_commune'].value_counts())

Index(['uid_adresse', 'cle_interop', 'commune_insee', 'commune_nom',
       'commune_deleguee_insee', 'commune_deleguee_nom', 'voie_nom',
       'lieudit_complement_nom', 'numero', 'suffixe', 'position', 'x', 'y',
       'long', 'lat', 'cad_parcelles', 'source', 'date_der_maj',
       'certification_commune'],
      dtype='object')
certification_commune
1    471170
0    293978
Name: count, dtype: int64


# Création de nouveaux liens bâtiments <> adresses

In [2]:
certified_df = df[df['certification_commune'] == 1]    
certified_df.reset_index(drop=True)
print(f"Adresses certifiées : {len(certified_df)}")

Adresses certifiées : 471170


In [3]:
def address_already_exists(bdg, row):
    already_exists = False
    for address in bdg.addresses_read_only.all():
        already_exists = already_exists or (address.city_name.lower() == row['commune_nom'].lower() and address.street.lower() == row['voie_nom'].lower() and int(address.street_number) == row['numero'])
    return already_exists

In [7]:
from batid.models import Building

new_links = set()
old_link_new_ban_id = set()
to_scan = 200000
c = 0
start_at = 100001
for index, row in certified_df.iterrows():

    c += 1

    if c < start_at:
        continue

    point = Point(row["long"], row["lat"], 4326)

    bdg = (Building.objects
           .filter(shape__intersects=point, is_active=True)
           .exclude(addresses_id__contains=[row['cle_interop']])
           .prefetch_related("addresses_read_only"))

    if bdg:
        already_exists = address_already_exists(bdg[0], row)
        couple = (bdg[0].rnb_id, row['cle_interop'])
        if already_exists:
            old_link_new_ban_id.add(couple)
        else:
            new_links.add(couple)


    if c % 100 == 0:
        print('--')
        print(f"row {c}")
        print(f"{len(new_links)} new links so far and {len(old_link_new_ban_id)} existing links but with a new ban id")


    if c >= to_scan:
        break


print(f"found {len(new_links)}, scanning {to_scan} rows")

--
row 100100
0 new links so far and 0 existing links but with a new ban id
--
row 100200
0 new links so far and 0 existing links but with a new ban id
--
row 100300
0 new links so far and 0 existing links but with a new ban id
--
row 100400
1 new links so far and 0 existing links but with a new ban id
--
row 100500
1 new links so far and 0 existing links but with a new ban id
--
row 100600
5 new links so far and 0 existing links but with a new ban id
--
row 100700
6 new links so far and 0 existing links but with a new ban id
--
row 100800
6 new links so far and 0 existing links but with a new ban id
--
row 100900
10 new links so far and 0 existing links but with a new ban id
--
row 101000
15 new links so far and 6 existing links but with a new ban id
--
row 101100
15 new links so far and 7 existing links but with a new ban id
--
row 101200
15 new links so far and 7 existing links but with a new ban id
--
row 101300
16 new links so far and 7 existing links but with a new ban id



KeyboardInterrupt



## Eval 2 : utilisation avec les parcelles

On veut pour chaque ligne : 
- vérifier si le point tombe sur le bâtiment ou très proche 
- si ce n'est pas le cas, vérifier si il tombe sur une parcelle
- si il tombe sur une parcelle, prendre tous les bâtiments supérieur à 35m2 d'emprise au sol étant sur cette parcelle
- si il y a un seul bâtiment ET qu'il n'a pas l'adresse concernée ET que le script n'attribue pas d'autres adresse à ce bâtiment



In [13]:
from django.contrib.gis.geos import MultiPoint
from collections import defaultdict
import time

def display_addr(row):

    # print(row)

    addr_infos = []

    if str(row["numero"]) != "nan":
        addr_infos.append(str(row["numero"]))

    addr_infos.append(row["voie_nom"])
    addr_infos.append(row["commune_nom"])

    print(row["cle_interop"])
    print(" ".join(addr_infos))
    print(row["cad_parcelles"])


q = """
    WITH points AS (
        SELECT unnest(%(lng_list)s) AS lng,
            unnest(%(lat_list)s) AS lat,
            unnest(%(cle_interop_list)s) AS cle_interop
    )
    SELECT
        p.id AS plot_id,
        b.rnb_id AS rnb_id,
        st_area(st_intersection(b.shape, p.shape)) / st_area(b.shape) AS bdg_cover_ratio,
        st_area(b.shape::geography) AS bdg_area,
        b.addresses_id AS bdg_addresses,
        pt.lng,
        pt.lat,
        pt.cle_interop
    FROM points pt
    JOIN LATERAL (
        SELECT * FROM batid_plot
        WHERE shape && st_expand(st_setsrid(st_makepoint(pt.lng, pt.lat), 4326), %(buffer_size)s)
        AND st_intersects(shape, st_buffer(st_setsrid(st_makepoint(pt.lng, pt.lat), 4326), %(buffer_size)s))
    ) p ON TRUE
    LEFT JOIN batid_building AS b ON b.shape && p.shape
        AND st_intersects(p.shape, b.shape)
    WHERE ST_GeometryType(b.shape) != 'ST_Point';
    """

new_links = set()
old_link_new_ban_id = set()
batch_size = 1000
to_scan = 1000000
batches_handled = 0

batches = certified_df.reset_index(drop=True)
batches = batches.groupby(batches.index // batch_size)

start_time = time.time()
with connection.cursor() as cursor:
    for i, batch in batches:
        batches_handled += 1
        if batches_handled * batch_size > to_scan:
            break

        points = [(row["long"], row["lat"]) for _, row in batch.iterrows()]
        point_to_cle_interop = {
            Point(row["long"], row["lat"], srid=4326): row["cle_interop"]
            for _, row in batch.iterrows()
        }

        # ##################
        # First we test with the most precise method
        excluded_addresses = [row["cle_interop"] for _, row in batch.iterrows()]
        points_list = [Point(lng, lat, srid=4326) for lng, lat in points]
        multi_point = MultiPoint(points_list, srid=4326)

        bdgs = Building.objects.filter(
            shape__intersects=multi_point,
            is_active=True
        ).exclude(addresses_id__overlap=excluded_addresses)

        cles_interop_already_linked = set()
        for building in bdgs:
            for point, cle_interop in point_to_cle_interop.items():
                if building.shape.intersects(point):
                    row = batch[batch["cle_interop"] == cle_interop].iloc[0]
                    already_exists = address_already_exists(building, row)
                    link = (building.rnb_id, cle_interop)
                    cles_interop_already_linked.add(cle_interop)
                    if already_exists:
                        old_link_new_ban_id.add(link)
                    else:
                        new_links.add(link)

        # ##################
        # If we found no precise match, we continue with a less precise method using plots
        other_rows = batch[~batch["cle_interop"].isin(cles_interop_already_linked)]
        if len(other_rows) > 0:
            points = [(row["long"], row["lat"]) for _, row in other_rows.iterrows()]
            cle_interop_list = [row["cle_interop"] for _, row in other_rows.iterrows()]
            params = {
                "lng_list": [p[0] for p in points],
                "lat_list": [p[1] for p in points],
                "cle_interop_list": cle_interop_list,
                "buffer_size": 0.00002 # 0.00002 seems to be a good value to check if the point is close to many plots
            }
            # Print raw sql
            cursor.execute(q, params)
            plots = dictfetchall(cursor, q, params)

            # pprint(plots)

            # The bdg matching using plots is tricky. We have to be very conservative.
            # We have many ambiguous situations to filter out:
            # - many obviously independant buildings with different addresses on the same plot. We filter them using building area
            # - address very close to many plots (we filter out the case using a small on address point and removing cases where we intersect multiple plots)
            # - building is not covered enough by a plot
            # we handle those cases along the script

            # Count unique plots and big buildings covered by the plot they intersect
            plots_by_cle_interop = defaultdict(list)
            for plot in plots:
                plots_by_cle_interop[plot['cle_interop']].append(plot)

            # Process each point separately
            for _, row in other_rows.iterrows():
                plots = plots_by_cle_interop[row['cle_interop']]

                # Get unique plots for this point
                plots_ids = {plot["plot_id"] for plot in plots}
                covered_enough_big_bdgs = []

                for plot in plots:
                    if plot["rnb_id"] and plot["bdg_cover_ratio"] > 0.75 and plot["bdg_area"] >= 35:
                        covered_enough_big_bdgs.append({
                            'rnb_id': plot['rnb_id'],
                            'addresses': plot['bdg_addresses']
                        })

                # If we have many plots, it is too ambiguous, we skip
                if len(plots_ids) > 1:
                    continue

                # If we have none or many big buildings, it is too ambiguous, we skip
                if len(covered_enough_big_bdgs) != 1:
                    continue

                # We now have one building in one plot
                last_bdg = covered_enough_big_bdgs[0]

                # We check the cle_interop is not already linked to the bdg
                if row['cle_interop'] in last_bdg['addresses']:
                    continue

                bdg = Building.objects.filter(rnb_id=last_bdg['rnb_id']).prefetch_related("addresses_read_only").get()
                already_exists = address_already_exists(bdg, row)
                link = (last_bdg["rnb_id"], row['cle_interop'])
                if already_exists:
                    old_link_new_ban_id.add(link)
                else:
                    new_links.add(link)

        print(f"batch : {batches_handled}, found {len(new_links)} new links so far and {len(old_link_new_ban_id)} existing links but with a new ban id on {batches_handled * batch_size} inspected rows")


# 26913886 addresses in adresses-france.csv
total_addresses = 26913886
execution_time = time.time() - start_time
time_for_all_addresses = execution_time / ((batches_handled - 1) * batch_size) * total_addresses
print(f"found {len(new_links)} new bdg addr links, scanning {(batches_handled - 1) * batch_size} BAL rows")
print(f"Total execution time : {execution_time:.2f} secondes, time for all addresses: {(time_for_all_addresses / 60 / 60 / 24):.2f} jours")


# print(matches)

batch : 1, found 411 new links so far and 0 existing links but with a new ban id on 1000 inspected rows
batch : 2, found 454 new links so far and 2 existing links but with a new ban id on 2000 inspected rows
batch : 3, found 481 new links so far and 6 existing links but with a new ban id on 3000 inspected rows
batch : 4, found 529 new links so far and 11 existing links but with a new ban id on 4000 inspected rows
batch : 5, found 563 new links so far and 33 existing links but with a new ban id on 5000 inspected rows
batch : 6, found 656 new links so far and 40 existing links but with a new ban id on 6000 inspected rows
batch : 7, found 703 new links so far and 45 existing links but with a new ban id on 7000 inspected rows
batch : 8, found 910 new links so far and 56 existing links but with a new ban id on 8000 inspected rows
batch : 9, found 1133 new links so far and 65 existing links but with a new ban id on 9000 inspected rows
batch : 10, found 1168 new links so far and 78 existing l

# Indice pour nouveaux bâtiments

In [10]:
position_counts = certified_df['position'].value_counts()
print(position_counts)

position
entrée                399350
logement               27291
bâtiment               18245
segment                11615
parcelle                7601
délivrance postale      3727
service technique        145
cage d’escalier           53
Name: count, dtype: int64


In [11]:
bdg_df = df[df["position"] == "bâtiment"]
print(f"Adresses certifiées de type bâtiments : {len(bdg_df)}")

Adresses certifiées de type bâtiments : 19037


In [12]:
from batid.services.closest_bdg import get_closest_from_point


to_print = 150
printed = 0
wo_rnb_bdgs = 0
c = 0
for index, row in bdg_df.iterrows():

    c += 1
    
    if c % 10 == 0:
        print(c)
    
    bdgs = get_closest_from_point(row["lat"],row["long"],10)[:2]
    
    if len(bdgs) == 0:
        
        wo_rnb_bdgs += 1
        
        

        # print(row)
        
        address_infos = [
            str(row['numero']),
            row['voie_nom'],
            row['commune_nom']
        ]
        
        if printed < to_print:
            printed += 1
            print('----')        
            print(row['cle_interop'])
            print(" ".join(address_infos))
            print(f"{row['lat']},{row['long']}")
        
        if wo_rnb_bdgs % 10 == 0:
            print(f"so far, found {wo_rnb_bdgs} bal row without RNB bdg")

    
print(f"finally found {wo_rnb_bdgs} bal row without RNB bdg")

    

----
33002_l62z72_00010
10 Route de la Réole Aillas
44.481977,-0.065006
----
33002_l62z72_00020
20 Route de la Réole Aillas
44.511027,-0.052362
----
33002_l62z72_00033
33 Route de la Réole Aillas
44.500943,-0.058539
----
33002_on6ia4_00001
1 Rue du Bois Majou Aillas
44.512026,-0.052731
10
----
33553_nlw27d_00002
2 Rue Jean Laviladie Virsac
45.026554,-0.441185
----
33553_nlw27d_00004
4 Rue Jean Laviladie Virsac
45.02657,-0.441463
20
30
40
----
33553_0470_00018
18 Route de la Mairie Virsac
45.021155,-0.443208
50
----
33553_0470_00028
28 Route de la Mairie Virsac
45.021236,-0.444026
60
70
----
33553_0470_00127
127 Route de la Mairie Virsac
45.029349,-0.440939
80
90
100
110
120
130
140
150
160
----
33553_0050_00038_a
38 Route des Chateaux Virsac
45.030049,-0.446547
so far, found 10 bal row without RNB bdg
----
33553_0050_00038_b
38 Route des Chateaux Virsac
45.02981,-0.446566
----
33553_0050_00040_a
40 Route des Chateaux Virsac
45.030109,-0.446312
170
180
190
200
210
220
230
240
250
----
3

KeyboardInterrupt: 

# Evaluation France

In [13]:
chunksize = 10000
count = 0
for chunk in pd.read_csv('adresses-france.csv', sep=";", chunksize=chunksize):
    count += chunk[(chunk["certification_commune"] == 1) & (chunk["position"] == "bâtiment")].shape[0]


print(count)

FileNotFoundError: [Errno 2] No such file or directory: 'adresses-france.csv'